In [ ]:
!pip install pinecone-client
from pinecone import Pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install pandas tqdm pinecone-client openai transformers python-dotenv peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 15.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
import pinecone
import pandas as pd
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv
from tqdm.auto import tqdm
from peft import PeftModel
from openai import OpenAI

In [ ]:
from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(pinecone_api_key)
index = pc.Index('cs224n-shorter')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
checkpoint_path = os.path.join('/content/drive/MyDrive', 'checkpoint-1250')


In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
community_summary_df = pd.read_csv('community_summaries_shortened.csv')

In [ ]:
community_summary_df

,ID,Title,Granularity,Topic,Text
0,The_Tortured_Poets_Department_Detailed_Product...,The Tortured Poets Department,Detailed,Production Details,"The Tortured Poets Department, Taylor Swift's ..."
1,The_Tortured_Poets_Department_Detailed_Collabo...,The Tortured Poets Department,Detailed,Collaborations and Contributions,"The Tortured Poets Department, Taylor Swift's ..."
2,The_Tortured_Poets_Department_Detailed_Commerc...,The Tortured Poets Department,Detailed,Commercial Success,"The Tortured Poets Department, Taylor Swift's ..."
3,The_Tortured_Poets_Department_Critical_Reception,The Tortured Poets Department,Detailed,Critical Reception,"The Tortured Poets Department, Taylor Swift's ..."
4,Oppenheimer_Detailed_Production_Details,Oppenheimer,Detailed,Production Details,Directed by Christopher Nolan and released by ...
5,Oppenheimer_Historical_Accuracy,Oppenheimer,Detailed,Historical Accuracy,"Oppenheimer, directed by Christopher Nolan and..."
6,Oppenheimer_Detailed_Character_Study,Oppenheimer,Detailed,Oppenheimer Character Study,"Oppenheimer, directed by Christopher Nolan and..."
7,Oppenheimer_Detailed_Ensemble_Cast_Performances,Oppenheimer,Detailed,Ensemble Cast and Performances,"Oppenheimer features a stellar ensemble cast, ..."
8,Oppenheimer_Detailed_Ethical_Moral_Dilemmas,Oppenheimer,Detailed,Ethical and Moral Dilemmas,"Directed by Christopher Nolan, Oppenheimer del..."
9,Oppenheimer_Detailed_Critical_Reception,Oppenheimer,Detailed,Critical Reception,"Oppenheimer, directed by Christopher Nolan and..."


In [ ]:
!pip install openai
import openai

In [ ]:
from openai import OpenAI

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')


In [ ]:
client = OpenAI(api_key=openai_api_key)


In [ ]:
!pip install accelerate

After embedding the community summaries, we can try doing GRAFT. Load FT model and perform RAG on community summaries.

In [ ]:
# Load fine-tuned model
base_model_id = "microsoft/phi-2"

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_auth_token=hf_token
)
ft_model = PeftModel.from_pretrained(base_model, checkpoint_path, torch_dtype=torch.float16, is_trainable=False)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.9 MB/s eta 0:00:00


In [ ]:
def retrieve_top_k(question, k=3):
    question_embedding = get_embedding(question)
    res = index.query(vector=question_embedding, top_k=k, include_metadata=True)
    return [(item['id'], item['metadata']['Title'], item['metadata']['Topic'], item['metadata']['Text']) for item in res['matches']]

def gen(model, prompt, maxlen=100, sample=True):
    toks = eval_tokenizer(prompt, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample, num_return_sequences=1, temperature=0.1, num_beams=1, top_p=0.95).to('cpu')
    return eval_tokenizer.batch_decode(res, skip_special_tokens=True)

def truncate_context(context, question, max_tokens):
    # Calculate the number of tokens used by the question and prompt
    prompt_length = len(eval_tokenizer(question)['input_ids']) + len(eval_tokenizer("Answer the question with given context:\nContext:")['input_ids'])

    # Truncate the context to fit within the remaining token limit
    max_context_length = max_tokens - prompt_length
    context_tokens = eval_tokenizer(context, return_tensors="pt")['input_ids'][0]

    if len(context_tokens) > max_context_length:
        truncated_context = eval_tokenizer.decode(context_tokens[:max_context_length])
    else:
        truncated_context = context

    return truncated_context

def rag_inference(question, max_input_length=2048):
    retrieved_documents = retrieve_top_k(question)
    retrieved_metadata = [(doc[0], doc[1], doc[2]) for doc in retrieved_documents]  # ID, Title, Topic
    full_context = " ".join([doc[3] for doc in retrieved_documents])

    truncated_context = truncate_context(full_context, question, max_input_length)
    input_text = f"Answer the question with given context: {question}\nContext: {truncated_context}\nAnswer:"

    generated = gen(model, input_text, 500)

    # Print the full generated output for debugging
    print("Full generated output:", generated)

    # Get the generated text and try to isolate the answer
    full_generated_text = generated[0].replace(input_text, '').strip()

    # Attempt to split by newline and find the most probable answer
    answer_lines = full_generated_text.split('\n')

    # Use the first line that does not include the context as the answer
    for line in answer_lines[1:]:
        print(line)
        if line.strip() and not line.startswith("Context:"):
            answer = line.strip()
            break

    # Fallback if the extraction is incorrect
    if not answer or "Question" in answer:
        print('Entered fallback, current answer:', answer)
        answer = "Answer not found or incorrect extraction"

    print("Extracted Answer:", answer)
    return answer, truncated_context, retrieved_metadata


In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from tqdm import tqdm

def evaluate_global_questions(global_questions_csv):
    # Load the global questions CSV
    global_questions_df = pd.read_csv(global_questions_csv)

    results = []
    correct_summaries = 0

    for index, row in tqdm(global_questions_df.iterrows(), total=global_questions_df.shape[0]):
        question = row['Question']
        expected_summary = row['Related Summary']
        answer, context, retrieved_metadata = rag_inference(question)

        # Format the retrieved summaries
        retrieved_summaries = [f"{item[2]} - {item[1]}" for item in retrieved_metadata]  # Format as Topic - Title

        # Check if the expected summary is in the retrieved summaries
        is_correct = any(expected_summary.lower().strip() in summary.lower().strip() for summary in retrieved_summaries)

        results.append({
            'Question': question,
            'Generated Answer': answer,
            'Context': context,
            'Expected Summary': expected_summary,
            'Retrieved Summaries': retrieved_summaries,
            'Is Summary Correct': is_correct
        })

        if is_correct:
            correct_summaries += 1
        break;

    accuracy = correct_summaries / len(global_questions_df)
    print(f"Accuracy: {accuracy}")

    results_df = pd.DataFrame(results)
    results_df.to_csv('evaluation_results.csv', index=False)
    print("Results saved to evaluation_results.csv")

# Assuming rag_inference is defined elsewhere


In [ ]:
global_questions_csv = 'global questions - graphrag.csv'

In [ ]:
evaluate_global_questions(global_questions_csv)

FileNotFoundError: [Errno 2] No such file or directory: 'global questions - graphrag.csv'